In [34]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import torch
import torch.utils.data as Data
import torch.nn as nn

import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.optim import lr_scheduler

datasets = pd.read_csv('SFEW.csv', skiprows=1,
                       names=['image', 'label', 'LPQ1', 'LPQ2', 'LPQ3', 'LPQ4', 'LPQ5', 'PHOG1', 'PHOG2', 'PHOG3',
                              'PHOG4', 'PHOG5'])
# print(datasets.label.value_counts())

tr, test = train_test_split(datasets, stratify=datasets.label, test_size=0.2)
# print(tr.label.value_counts())
# print(test.label.value_counts())
tr, val = train_test_split(tr, stratify=tr.label, test_size=0.2)
# print(val.label.value_counts())

# traning set
tr_features_LPQ = [[lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5] for
                   lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5 in
                   zip(tr.LPQ1, tr.LPQ2, tr.LPQ3, tr.LPQ4, tr.LPQ5, tr.PHOG1, tr.PHOG2, tr.PHOG3, tr.PHOG4, tr.PHOG5)]
# tr_features_LPQ = [[lp1,lp2,lp3,lp4,lp5] for lp1,lp2,lp3,lp4,lp5  in zip(tr.LPQ1,tr.LPQ2,tr.LPQ3,tr.LPQ4,tr.LPQ5)]
tr_features_PHOG = [[ph1, ph2, ph3, ph4, ph5] for ph1, ph2, ph3, ph4, ph5 in
                    zip(tr.PHOG1, tr.PHOG2, tr.PHOG3, tr.PHOG4, tr.PHOG5)]
tr_labels = [l - 1 for l in tr.label]
# print(tr_labels)
# from list to tensor
tr_features_LPQ_tensor = torch.FloatTensor(tr_features_LPQ)
tr_features_PHOG_tensor = torch.FloatTensor(tr_features_PHOG)
tr_labels_tensor = torch.FloatTensor(tr_labels)
# train data
torch_dataset_train_LPQ = Data.TensorDataset(tr_features_LPQ_tensor, tr_labels_tensor)
torch_dataset_train_PHOG = Data.TensorDataset(tr_features_PHOG_tensor, tr_labels_tensor)

# validation set
val_features_LPQ = [[lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5] for
                   lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5 in
                   zip(val.LPQ1, val.LPQ2, val.LPQ3, val.LPQ4, val.LPQ5, val.PHOG1, val.PHOG2, val.PHOG3, val.PHOG4, val.PHOG5)]
# val_features_LPQ = [[lp1, lp2, lp3, lp4, lp5] for lp1, lp2, lp3, lp4, lp5 in
#                     zip(val.LPQ1, val.LPQ2, val.LPQ3, val.LPQ4, val.LPQ5)]
val_features_PHOG = [[ph1, ph2, ph3, ph4, ph5] for ph1, ph2, ph3, ph4, ph5 in
                     zip(val.PHOG1, val.PHOG2, val.PHOG3, val.PHOG4, val.PHOG5)]
val_labels = [l - 1 for l in val.label]
# from list to tensor
val_features_LPQ_tensor = torch.FloatTensor(val_features_LPQ)
val_features_PHOG_tensor = torch.FloatTensor(val_features_PHOG)
val_labels_tensor = torch.FloatTensor(val_labels)
# validation data
torch_dataset_val_LPQ = Data.TensorDataset(val_features_LPQ_tensor, val_labels_tensor)
torch_dataset_val_PHOG = Data.TensorDataset(val_features_PHOG_tensor, val_labels_tensor)

# test set
test_features_LPQ = [[lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5] for
                     lp1, lp2, lp3, lp4, lp5, ph1, ph2, ph3, ph4, ph5 in
                     zip(test.LPQ1, test.LPQ2, test.LPQ3, test.LPQ4, test.LPQ5, test.PHOG1, test.PHOG2, test.PHOG3,
                         test.PHOG4, test.PHOG5)]
# test_features_LPQ = [[lp1,lp2,lp3,lp4,lp5] for lp1,lp2,lp3,lp4,lp5 in zip(test.LPQ1,test.LPQ2,test.LPQ3,test.LPQ4,test.LPQ5)]
test_features_PHOG = [[ph1, ph2, ph3, ph4, ph5] for ph1, ph2, ph3, ph4, ph5 in
                      zip(test.PHOG1, test.PHOG2, test.PHOG3, test.PHOG4, test.PHOG5)]
test_labels = [l - 1 for l in test.label]
# from list to tensor
test_features_LPQ_tensor = torch.FloatTensor(test_features_LPQ)
test_features_PHOG_tensor = torch.FloatTensor(test_features_PHOG)
test_labels_tensor = torch.FloatTensor(test_labels)
# test data
torch_dataset_test_LPQ = Data.TensorDataset(test_features_LPQ_tensor, test_labels_tensor)
torch_dataset_test_PHOG = Data.TensorDataset(test_features_PHOG_tensor, test_labels_tensor)

batch_size = 540
num_workers = 0

train_loader = Data.DataLoader(torch_dataset_train_LPQ, batch_size=batch_size, num_workers=num_workers, shuffle=True)
val_loader = Data.DataLoader(torch_dataset_val_LPQ, batch_size=batch_size, num_workers=num_workers, shuffle=True)
test_loader = Data.DataLoader(torch_dataset_test_LPQ, batch_size=batch_size, num_workers=num_workers, shuffle=True)


# model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(10, 40)
        self.batchnorm1d1 = nn.BatchNorm1d(40)
        self.fc2 = nn.Linear(40, 80)
        self.batchnorm1d2 = nn.BatchNorm1d(80)
        self.fc3 = nn.Linear(80, 160)
        self.batchnorm1d3 = nn.BatchNorm1d(160)
        self.fc4 = nn.Linear(160, 7)
        self.dropout = nn.Dropout(0.5)
        self.sof = nn.Softmax()

    #         self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1d1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batchnorm1d2(x)
        x = F.relu(x)
        # x = self.dropout(x)
        x = self.fc3(x)
        x = F.relu(x)
        # x = self.dropout(x)
        x = self.batchnorm1d3(x)
        x = F.relu(x)
        # x = self.dropout(x)
        x = self.fc4(x)
        x = F.relu(x)
        y = F.softmax(x)
        return x, y


model_conv = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_conv.parameters(), lr=0.0004)
# optimizer = optim.SGD(model_conv.parameters(), lr=0.01, momentum=0.5)
EPOCH = 20001

for epoch in range(EPOCH):
    for step, (data, target) in enumerate(train_loader):  # 分配 batch data, normalize x when iterate train_loader
        flag = True
        for e in data.numpy():
            if any(np.isnan(e)):
                flag = False
                break
        if not flag:
            continue
        model_conv.train()
        output = model_conv(data)  # output
        loss = criterion(output[1], target.long())  # cross entropy loss
        optimizer.zero_grad()  # clear gradients for this training step
        loss.backward()  # backpropagation, compute gradients
        optimizer.step()  # apply gradients
        if epoch % 1000 == 0:
            test_output = model_conv(val_features_LPQ_tensor)
            # print(test_output)
            pred_y = torch.max(test_output[1], 1)[1].data.numpy()
            # print(pred_y)
            # print(val_labels_tensor.data.numpy().astype(int))
            accuracy = float((pred_y == val_labels_tensor.data.numpy()).astype(int).sum()) / float(val_labels_tensor.size(0))
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.numpy(), '| val accuracy: %.2f' % accuracy)



KeyboardInterrupt: 

In [17]:
list1 = [1,2,3]
print(list1*3)

[1, 2, 3, 1, 2, 3, 1, 2, 3]


In [ ]:
# model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(40, 40)
        self.batchnorm1d1 = nn.BatchNorm1d(40)
        self.fc2 = nn.Linear(40,80)
        self.batchnorm1d2 = nn.BatchNorm1d(80)
        self.fc3 = nn.Linear(80,160)
        self.batchnorm1d3 = nn.BatchNorm1d(160)
        self.fc4 = nn.Linear(160,320)
        self.batchnorm1d4 = nn.BatchNorm1d(320)
        self.fc5 = nn.Linear(320,160)
        self.batchnorm1d5 = nn.BatchNorm1d(160)
        self.fc6 = nn.Linear(160,7)
        self.dropout = nn.Dropout(0.5)
        self.sof = nn.Softmax()
        self.sig = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.batchnorm1d1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = self.batchnorm1d2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = self.batchnorm1d3(x)
        x = F.relu(x)
        x = self.fc4(x)
        x = self.batchnorm1d4(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc5(x)
        x = self.batchnorm1d5(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.fc6(x)
        y = self.sof(x)
        return x,y
    
loss 1.1654